# Energy Statistics

Simple notebook to extract general stats about the energy production and consumption.

In [1]:
import aioinflux
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
kwargs = {
    'host': '192.168.178.47',
    'port': os.getenv('INFLUXDB_PORT'),
    'username': os.getenv('INFLUXDB_USER'),
    'password': os.getenv('INFLUXDB_PASS'),
    'db': os.getenv('INFLUXDB_DB'),
    'output': 'dataframe'
}

In [3]:
today = datetime.today().date()
print(f"Today is {today}")

Today is 2021-04-26


In [4]:
measurement = 'electricity_energy_watthour'
async with aioinflux.InfluxDBClient(**kwargs) as ifclient:
    df_energy = await ifclient.query(f"""
        SELECT * FROM {measurement}
    """)

In [5]:
# consider only last year
d = df_energy[today-pd.DateOffset(years=1):today]

(
    d
    .groupby(d.index.date)
    .sum()
    .quantile(q=[0.1, 0.25, 0.5, 0.75, 0.9])
    *0.02 # round to 50
).round(decimals=0)/0.02

/Users/christoph.sawade/Library/Python/3.8/lib/python/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


,Consumption,FeedIn,Production,Purchased,SelfConsumption
0.10,5150.0,50.0,1000.0,2300.0,850.0
0.25,6050.0,1600.0,3450.0,2800.0,1650.0
0.50,7200.0,6850.0,9650.0,4000.0,2700.0
0.75,8400.0,12650.0,16600.0,6000.0,3800.0
0.90,9950.0,16300.0,20100.0,8600.0,4700.0


In [6]:
measurement = 'electricity_power_watt'
async with aioinflux.InfluxDBClient(**kwargs) as ifclient:
    df_power = await ifclient.query(f"""
        SELECT * FROM {measurement}
    """)

In [7]:
# consider only last year
d = df_power[today-pd.DateOffset(years=1):today]

# ignore lower half (quiescent current)
d[d<=d.quantile(q=0.5, axis=0)]=np.nan

(
    d
    .quantile(q=[0.1, 0.25, 0.5, 0.75, 0.9], axis=0)
    *0.02 # round to 50
).round(decimals=0)/0.02

/Users/christoph.sawade/Library/Python/3.8/lib/python/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)
<ipython-input-7-aff87223671e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[d<=d.quantile(q=0.5, axis=0)]=np.nan
/Users/christoph.sawade/Library/Python/3.8/lib/python/site-packages/pandas/core/frame.py:3215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

,Consumption,FeedIn,Production,Purchased,SelfConsumption
0.10,200.0,50.0,350.0,150.0,50.0
0.25,250.0,200.0,550.0,200.0,150.0
0.50,250.0,700.0,1100.0,250.0,200.0
0.75,350.0,1500.0,1850.0,300.0,250.0
0.90,950.0,2150.0,2450.0,500.0,350.0
